In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from Bio import SeqIO
import difflib
import random
from scipy import stats
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
import warnings
warnings.filterwarnings('ignore')

In [2]:
#find out the number of times that the keyword "transcriptional" show in the description
n=0
t=0
with open('ecoli_sequence.txt', mode='r') as handle:
    for record in SeqIO.parse(handle, 'fasta'):
        t+=1
        identifier = record.id
        description = record.description
        sequence = record.seq
        p_sequence=sequence.translate(to_stop=True)
        if 'transcriptional' in description:
            n+=1
print(n)
print(t-n)
print(t)

291
4066
4357


In [3]:
#create the dictionary called "ret_lib" shows the keywords that shows for the most times
import operator
word_count={}
with open('ecoli_sequence.txt', mode='r') as handle:
    for record in SeqIO.parse(handle, 'fasta'):
        t+=1
        identifier = record.id
        description = record.description
        sequence = record.seq
        p_sequence=sequence.translate(to_stop=True)
        start=description.find('protein=')
        end=description.find('] [protein_id=')
        prot=description[start:end][8:]
        prot_li=prot.split(' ')
        for w in prot_li:
            if w in word_count:
                word_count[w]=word_count[w]+1
            else:
                word_count[w]=1
                
key_list = list(word_count.keys()) 
val_list = list(word_count.values())
ret_lib={}
for i in range(50):
    max_value = max(val_list)
    max_index = val_list.index(max_value)
    ret_lib[key_list[max_index]]=max_value
    val_list[max_index]=0

In [4]:
ret_lib

{'protein': 1432,
 'putative': 820,
 'subunit': 453,
 'transcriptional': 291,
 'transporter': 283,
 'membrane': 276,
 'DNA-binding': 260,
 'domain-containing': 223,
 'prophage;': 212,
 'regulator': 201,
 'ABC': 195,
 'synthase': 125,
 'family': 124,
 'binding': 119,
 'uncharacterized': 117,
 '[pseudo=true]': 115,
 '[gbkey=CDS': 115,
 'dehydrogenase': 101,
 'reductase': 101,
 'inner': 100,
 'kinase': 92,
 'periplasmic': 86,
 'lipoprotein': 83,
 'dual': 81,
 'factor': 78,
 'DNA': 78,
 'repressor': 75,
 'conserved': 73,
 'system': 72,
 'enzyme': 70,
 'ribosomal': 66,
 'of': 65,
 'component': 65,
 'symporter': 62,
 'oxidoreductase': 62,
 'ATP': 61,
 'outer': 59,
 'A': 52,
 'activator': 51,
 'methyltransferase': 51,
 'PTS': 49,
 'fimbrial': 49,
 'chaperone': 45,
 'hydrolase': 45,
 'and': 45,
 '2': 45,
 'acid': 45,
 'exporter': 44,
 '1': 43,
 'ligase': 42}

In [5]:
amino_acids={'A':11,'C':12,'D':13,'E':14,'F':15,'G':16,'H':17,'I':18,'K':19,
            'L':21,'M':22,'N':23,'P':24,'Q':25,'R':26,'S':27,'T':28,'V':29,'W':31,'Y':32}

In [6]:
#the list of possible 3-mers, totally 8000
kmers={}
n=0
for i in amino_acids:
    for j in amino_acids:
        for k in amino_acids:
            kmer=i+j+k
            kmers[kmer]=n
            n+=1

In [7]:
len(kmers)

8000

In [8]:
#the function to find the existing k-mers in a protein sequence
def find_kmer(seq):
    amino = {}
    l = len(seq)-2
    for i in range(l):
        three_mer = seq[i:i+3]
        if three_mer in amino:
            amino[three_mer] += 1
        else:
            amino[three_mer] = 1
    return amino

#the function to transfer the protein sequence to an array with number of k-mers as features
def seq_to_kmer(seq):
    ret=np.zeros((8000,), dtype=int)
    amino=find_kmer(seq)
    key_list = list(kmers.keys()) 
    val_list = list(kmers.values())
    for i in amino:
        index=kmers[i]
        value=amino[i]
        ret[index]=value
    return ret

In [9]:
seq_to_kmer('AAACDEFAAE')

array([1, 1, 0, ..., 0, 0, 0])

In [10]:
#the kernel used for SVM
def my_kernel(X, Y):
    
    return np.dot(X, Y.T)

In [11]:
#train the svm and logistic classifier for transcriptional function using the ecoli genome as training set
X=np.array([],'int64')
Y=np.array([],'int64')
n=0
t=0
with open('ecoli_sequence.txt', mode='r') as handle:
    for record in SeqIO.parse(handle, 'fasta'):
        identifier = record.id
        description = record.description
        sequence = record.seq
        p_sequence=sequence.translate(to_stop=True)
        if 'transcriptional' in description:
            s=seq_to_kmer(str(p_sequence))
            X=np.append(X, s)
            Y=np.append(Y,1)
            n=n+1
        else:
            rm=random.uniform(0,1)
            if (rm<0.072):
                t+=1
                s=seq_to_kmer(str(p_sequence))
                X=np.append(X, s)
                Y=np.append(Y,0)
X=np.reshape(X,(-1,8000))
clf = svm.SVC(kernel=my_kernel)
clf.fit(X, Y)
logreg = LogisticRegression(C=1e5)
logreg.fit(X, Y)

LogisticRegression(C=100000.0, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [12]:
ar=seq_to_kmer('MTEVRRRGRPGQAEPVAQKGAQALERGIAILQYLEKSGGSSSVSDISLNLDLPLSTTFRLLKVLQAADFVYQDSQLGWWHIGLGVFNVGAAYIHNRDVLSVAGPFMRRLMLLSGETVNVAIRNGNEAVLIGQLECKSMVRMCAPLGSRLPLHASGAGKALLYPLAEEELMSIILQTGLQQFTPTTLVDMPTLLKDLEQARELGYTVDKEEHVVGLNCIASAIYDDVGSVVAAISISGPSSRLTEDRFVSQGELVRDTARDISTALGLKAHP')
clf.predict([ar])

array([1])

In [13]:
#use the prokaryote "granu" as test set to test the accuracy of svm classifier for transcriptional function
n=0
ret=np.array([],'int64')
with open('granu_sequence.txt', mode='r') as handle:
    for record in SeqIO.parse(handle, 'fasta'):
        identifier = record.id
        description = record.description
        sequence = record.seq
        p_sequence=sequence.translate(to_stop=True)
        if 'transcriptional' in description:
            n+=1
            ar=seq_to_kmer(p_sequence)
            r=clf.predict([ar])
            ret=np.append(ret,r)
print('The accuracy (sensitivity) of svm classification is:'+str(sum(ret)/n))

The accuracy (sensitivity) of svm classification is:0.785


In [14]:
#use the prokaryote "granu" as test set to test the accuracy of logistic classifier for transcriptional function
n=0
ret=np.array([],'int64')
with open('granu_sequence.txt', mode='r') as handle:
    for record in SeqIO.parse(handle, 'fasta'):
        identifier = record.id
        description = record.description
        sequence = record.seq
        p_sequence=sequence.translate(to_stop=True)
        if 'transcriptional' in description:
            n+=1
            ar=seq_to_kmer(p_sequence)
            r=logreg.predict([ar])
            ret=np.append(ret,r)
print('The accuracy (sensitivity) of logistic classification is:'+str(sum(ret)/n))

The accuracy (sensitivity) of logistic classification is:0.75


In [15]:
#use the prokaryote "granu" as test set to test the accuracy of svm classifier for transcriptional function
n=0
ret=np.array([],'int64')
with open('granu_sequence.txt', mode='r') as handle:
    for record in SeqIO.parse(handle, 'fasta'):
        identifier = record.id
        description = record.description
        sequence = record.seq
        p_sequence=sequence.translate(to_stop=True)
        if 'transcriptional' not in description:
            n+=1
            ar=seq_to_kmer(p_sequence)
            r=1 - clf.predict([ar])
            ret=np.append(ret,r)
print('The accuracy (specificity) of svm classification is:'+str(sum(ret)/n))

The accuracy (specificity) of svm classification is:0.7070568510897729


In [16]:
#use the prokaryote "granu" as test set to test the accuracy of logistic classifier for transcriptional function
n=0
ret=np.array([],'int64')
with open('granu_sequence.txt', mode='r') as handle:
    for record in SeqIO.parse(handle, 'fasta'):
        identifier = record.id
        description = record.description
        sequence = record.seq
        p_sequence=sequence.translate(to_stop=True)
        if 'transcriptional' not in description:
            n+=1
            ar=seq_to_kmer(p_sequence)
            r=1 - logreg.predict([ar])
            ret=np.append(ret,r)
print('The accuracy (specificity) of logistic classification is:'+str(sum(ret)/n))

The accuracy (specificity) of logistic classification is:0.7626886145404664
